In [1]:
import json
import time
from time import sleep
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from pandas import *

In [78]:
#가게 이름, 가게 유형, 주소, 대표 메뉴 1개, 메뉴 설명, 리뷰 5개
def search_food(key):
    try:
        
        food_list = []
        
        url = 'https://map.naver.com/v5/search'
        driver = webdriver.Chrome()  # 드라이버 경로
        # driver = webdriver.Chrome('./chromedriver',chrome_options=options) # 크롬창 숨기기
        driver.get(url)
        key_word = key  # 검색어

        # css 찾을때 까지 10초대기
        def time_wait(num, code):
            try:
                wait = WebDriverWait(driver, num).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, code)))
            except:
                print(code, '태그를 찾지 못하였습니다.')
                driver.quit()
            return wait

        # frame 변경 메소드
        def switch_frame(frame):
            driver.switch_to.default_content()  # frame 초기화"
            driver.switch_to.frame(frame)  # frame 변경

        # 페이지 다운
        def page_down(num):
            body = driver.find_element(By.CSS_SELECTOR, 'body')
            body.click()
            for i in range(num):
                body.send_keys(Keys.PAGE_DOWN)

        # css를 찾을때 까지 10초 대기
        time_wait(10, 'div.input_box > input.input_search')

        # (1) 검색창 찾기
        search = driver.find_element(By.CSS_SELECTOR, 'div.input_box > input.input_search')
        search.send_keys(key_word)  # 검색어 입력
        search.send_keys(Keys.ENTER)  # 엔터버튼 누르기

        sleep(5)
        
        url_str = driver.current_url
        if url_str.find("isCorrectAnswer") == -1:
            print(url_str)
            print("결과 여러 개 존재(지점 또는 근접 지역)")
            
            search = driver.find_elements(By.ID, "searchIframe")
            # frame 변경
            switch_frame('searchIframe')
            sleep(2)
        
            #검색 결과 없음
            place_none = driver.find_elements(By.CSS_SELECTOR, '.FYvSc')
            
            for i in range(len(place_none)):
                if place_none[i].text == "조건에 맞는 업체가 없습니다.":
                    tmp = DataFrame({'가게명': "-", '가게 유형' :"-", '가게 주소': "-",
                            '메뉴 이름':"-", '메뉴 설명' : "-", "리뷰1": "-", "리뷰2": "-", "리뷰3": "-", "리뷰4": "-","리뷰5": "-"}, index = [0])
                    print("가게 없음")
                    return tmp
            
            tmp_shop = driver.find_elements(By.CSS_SELECTOR, '.YwYLL')
            if tmp_shop == []:
                tmp_shop = driver.find_elements(By.CSS_SELECTOR, '.place_bluelink')
            tmp_shop[0].click()
            
            sleep(5)
        
        else:
            print(url_str)
            print("결과 한 개")
            
        
        try:
        
            # (2) frame 변경
            switch_frame('entryIframe')
            #page_down(40)
            sleep(2)

            # 시작시간
            start = time.time()
            print('\n[크롤링 시작...]')
            
        except:
            tmp = DataFrame({'가게명': "-", '가게 유형' :"-", '가게 주소': "-",
                        '메뉴 이름':"-", '메뉴 설명' : "-", "리뷰1": "-", "리뷰2": "-", "리뷰3": "-", "리뷰4": "-","리뷰5": "-"}, index = [0])
            print("오류 예외 처리")
            return tmp
        
        shop_name = []
        shop_type = []
        shop_address = []
        menu_name = []
        menu_story = []
        review_1 = []
        review_2 = []
        review_3 = []
        review_4 = []
        review_5 = []
        

        #가게 이름
        names = driver.find_elements(By.CSS_SELECTOR, '.GHAhO')  # (3) 장소명
        shop_name.append(names[0].text)
        print(names[0].text)
        
        #가게 타입
        types = driver.find_elements(By.CSS_SELECTOR, '.lnJFt')  # (4) 장소 유형
        shop_type.append(types[0].text)
        print(types[0].text)
        
        #가게 주소
        address = driver.find_elements(By.CSS_SELECTOR, '.LDgIH') # (5) 주소
        shop_address.append(address[0].text)
        print(address[0].text)
        
        #버튼 정의
        home_list = driver.find_elements(By.CSS_SELECTOR, ".veBoZ") # (6) 리뷰
        
        #잠시 대기
        sleep(3)
        
        #배달 포장 여부
        order = driver.find_elements(By.XPATH, '/html/body/div[3]/div/div/div/div[2]/div[4]/div')
        order2 = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[4]/div/span/a')
        
        #print(order2)
        #print(order[0].text)
        if order[0].text == "주문준비중" or order[0].text == "주문\n배달" or order[0].text == "예약\n주문준비중" or order[0].text == "예약\n주문":
            order_result = 1 #배달 포장 가능
            print("주문 유형 1")
            
        elif (len(order2) != 0) and (order2[0].text == "주문"):
            order_result = 1 #배달 포장 가능
            print("주문 유형 2")
            
        else:
            order_result = 0 #배달 포장 불가능
            print("주문 유형 3")
        
        #메뉴 버튼 클릭
        for k in range(len(home_list)):
            if home_list[k].text == "메뉴":
                #print(home_list[k].text)
                home_list[k].click()
                sleep(3)
                break
            else:
                continue
        
        #배달 불가
        if order_result == 0:
            #상위 메뉴 1개 이름
            #print(menu_name)
            
            menu_btn_1 = driver.find_elements(By.CSS_SELECTOR, ".lPzHi") # (6-1) 메뉴
            
            if menu_btn_1 == []:
                menu_name.append("")
                print("대표 메뉴 없음")
            else: 
                menu_name.append(menu_btn_1[0].text)
                print(menu_btn_1[0].text)

            #메뉴 설명
            menu_btn_2 = driver.find_elements(By.CSS_SELECTOR, ".kPogF") # (6-2) 메뉴 설명
            if menu_btn_2 == []:
                menu_story.append("")
                print("메뉴 설명 없음")
                
            else:
                menu_story.append(menu_btn_2[0].text)
                print(menu_btn_2[0].text)
            
            sleep(3)
            
            
            try:
                #리뷰 버튼 클릭
                #메뉴 버튼 클릭
                for k in range(len(home_list)):
                    if home_list[k].text == "리뷰":
                        home_list[k].click()
                        sleep(3)
                        break
                    else:
                        continue
            except:
                tmp = DataFrame({'가게명': "-", '가게 유형' :"-", '가게 주소': "-",
                        '메뉴 이름':"-", '메뉴 설명' : "-", "리뷰1": "-", "리뷰2": "-", "리뷰3": "-", "리뷰4": "-","리뷰5": "-"}, index = [0])
                print("오류 예외 처리")
                return tmp
                
            
        #배달 가능
        else:
            sleep(3)
            #상위 메뉴 1개 이름
            #box = driver.find_elements(By.CSS_SELECTOR, ".info_detail")
            
            menu_btn_1 = driver.find_elements(By.CSS_SELECTOR, ".tit") # (6-1) 메뉴
            #print(menu_btn_1)
            menu_name.append(menu_btn_1[0].text)
            print(menu_btn_1[0].text)
            
            #메뉴 설명
            menu_btn_2 = driver.find_elements(By.CSS_SELECTOR, ".detail_txt") # (6-2) 메뉴 설명
            if menu_btn_2 == []:
                menu_story.append("")
                print("메뉴 설명 없음")
                
            else:
                menu_story.append(menu_btn_2[0].text)
                print(menu_btn_2[0].text)
            
            sleep(3)
            
            
            try:
                #리뷰 버튼 클릭
                #메뉴 버튼 클릭
                home_review_list = driver.find_elements(By.CSS_SELECTOR, '.txt')
                for k in range(len(home_review_list)):
                    if home_review_list[k].text == "리뷰":
                        home_review_list[k].click()
                        sleep(3)
                        break
                    else:
                        continue
            except:
                tmp = DataFrame({'가게명': "-", '가게 유형' :"-", '가게 주소': "-",
                        '메뉴 이름':"-", '메뉴 설명' : "-", "리뷰1": "-", "리뷰2": "-", "리뷰3": "-", "리뷰4": "-","리뷰5": "-"}, index = [0])
                print("오류 예외 처리")
                return tmp
            
        #리뷰 찾기
        review_btn = driver.find_elements(By.CSS_SELECTOR, ".t3JSf") # (7) 리뷰
        
        #이런 점이 좋았어요
        review_love = driver.find_elements(By.CSS_SELECTOR, ".place_section_header_title")


        
        review_love_yes = 0
        for i in range(len(review_love)):
            if review_love[i].text == "이런 점이 좋았어요\n안내":
                review_love_yes = 1
                print("리뷰 존재")
                break
            else:
                continue
                
        
        if review_love_yes == 0:
            tmp = DataFrame({'가게명': "-", '가게 유형' :"-", '가게 주소': "-",
                        '메뉴 이름':"-", '메뉴 설명' : "-", "리뷰1": "-", "리뷰2": "-", "리뷰3": "-", "리뷰4": "-","리뷰5": "-"}, index = [0])
        
        else:
            
            num_10_none = driver.find_elements(By.CSS_SELECTOR, ".zXXsw")
            
            if num_10_none != []:
                if num_10_none[0].text == "10명 이상 참여하면\n방문 키워드 통계가 공개됩니다!":
                    print("리뷰 10명 이하 참여")
                    tmp = DataFrame({'가게명': "-", '가게 유형' :"-", '가게 주소': "-",
                            '메뉴 이름':"-", '메뉴 설명' : "-", "리뷰1": "-", "리뷰2": "-", "리뷰3": "-", "리뷰4": "-","리뷰5": "-"}, index = [0])
                
            #리뷰 10명 이하 참여
            #if num_10_none[0].text == "10명 이상 참여하면\n방문 키워드 통계가 공개됩니다!":
            #    tmp = DataFrame({'가게명': "-", '가게 유형' :"-", '가게 주소': "-",
            #            '메뉴 이름':"-", '메뉴 설명' : "-", "리뷰1": "-", "리뷰2": "-", "리뷰3": "-", "리뷰4": "-","리뷰5": "-"}, index = [0])
            
            #리뷰 10명 이상 참여
            else:
                #리뷰 1
                real_review = review_btn[0].text.split('"')
                review_1.append(real_review[1])
                print(real_review[1])
                #리뷰 2
                real_review = review_btn[1].text.split('"')
                review_2.append(real_review[1])
                print(real_review[1])
                #리뷰 3
                real_review = review_btn[2].text.split('"')
                review_3.append(real_review[1])
                print(real_review[1])
                #리뷰 4
                real_review = review_btn[3].text.split('"')
                review_4.append(real_review[1])
                print(real_review[1])
                #리뷰 5
                real_review = review_btn[4].text.split('"')
                review_5.append(real_review[1])
                print(real_review[1])
            
                tmp = DataFrame({'가게명': shop_name, '가게 유형' :shop_type, '가게 주소': shop_address,
                                '메뉴 이름':menu_name, '메뉴 설명' : menu_story, "리뷰1": review_1, "리뷰2": review_2, "리뷰3": review_3, "리뷰4": review_4,"리뷰5": review_5})

        print(f'{shop_name} ...완료')

        sleep(1)

        print('[데이터 수집 완료]\n소요 시간 :', time.time() - start, "\n")
        driver.quit()  # 작업이 끝나면 창을 닫는다.
        
        return tmp
    
    except:
        tmp = DataFrame({'가게명': "-", '가게 유형' :"-", '가게 주소': "-",
                        '메뉴 이름':"-", '메뉴 설명' : "-", "리뷰1": "-", "리뷰2": "-", "리뷰3": "-", "리뷰4": "-","리뷰5": "-"}, index = [0])
        print("오류 예외 처리")
        return tmp

In [79]:

regions = ["종로구", "중구", "용산구", "성동구", "광진구", "동대문구", "중랑구", "성북구", "강북구", "도봉구", "노원구", "은평구", "서대문구", "마포구", "양천구", 
           "강서구", "구로구", "금천구", "영등포구", "동작구", "관악구", "서초구", "강남구", "송파구", "강동구"]

#len(regions)
for i in range(15, len(regions)):
    tmp = DataFrame(columns = ['가게명', '가게 유형', '가게 주소', '메뉴 이름', '메뉴 설명', '리뷰1', '리뷰2', '리뷰3', '리뷰4', '리뷰5'])
    filename = "./out_food/out_" + str(i) + "_food.csv"
    df = read_csv(filename, header = None)
    df.columns = ['가게명', '별점']
    
    #43, len(df)
    for j in range( len(df)):
        tp = df.loc[j]['가게명']
        key = regions[i] + " " + tp
        print(str(j) + "번째 가게")
        
        data = search_food(key)
        
        result2 = concat([tmp, data], ignore_index = True)
        tmp = result2

    #파일 저장
    filename = "./out_food_review/out_" + str(i) + "_food.csv"
    tmp.to_csv(filename, header = None, index = False)
    print(filename + "완료")


0번째 가게


The chromedriver version (128.0.6613.137) detected in PATH at c:\Users\kk21\Desktop\TRIPER\chromedriver.exe might not be compatible with the detected chrome version (129.0.6668.101); currently, chromedriver 129.0.6668.100 is recommended for chrome 129.*, so it is advised to delete the driver in PATH and retry


https://map.naver.com/p/search/%EA%B0%95%EC%84%9C%EA%B5%AC%20%EC%B6%98%20%EB%A7%88%EA%B3%A1%EC%A0%90/place/1785581002?c=15.00,0,0,0,dh&isCorrectAnswer=true
결과 한 개

[크롤링 시작...]
춘 마곡점
이자카야
서울 강서구 마곡중앙6로 70 2층 210호, 211호
주문 유형 3
모둠 사시미
10종류 이상의 구성으로 32피스 이상 제공
리뷰 존재
음식이 맛있어요
기본 안주가 좋아요
인테리어가 멋져요
친절해요
특별한 메뉴가 있어요
['춘 마곡점'] ...완료
[데이터 수집 완료]
소요 시간 : 13.477749586105347 

1번째 가게


The chromedriver version (128.0.6613.137) detected in PATH at c:\Users\kk21\Desktop\TRIPER\chromedriver.exe might not be compatible with the detected chrome version (129.0.6668.101); currently, chromedriver 129.0.6668.100 is recommended for chrome 129.*, so it is advised to delete the driver in PATH and retry


https://map.naver.com/p/search/%EA%B0%95%EC%84%9C%EA%B5%AC%20%EB%8B%A4%EC%98%A8%EB%8B%A4%20%EA%B9%80%EB%B0%A5%EA%B5%AD%EC%88%98/place/1747067359?c=15.00,0,0,0,dh&isCorrectAnswer=true
결과 한 개

[크롤링 시작...]
다온다 김밥국수
종합분식
서울 강서구 강서로47길 40 1층
주문 유형 3
잔치국수
직접 오래 우린 멸치 육수에, 당일 볶은 신선한 야채 고명을 얹어 맛있게 만들었습니다
리뷰 존재
음식이 맛있어요
재료가 신선해요
친절해요
가성비가 좋아요
혼밥하기 좋아요
['다온다 김밥국수'] ...완료
[데이터 수집 완료]
소요 시간 : 13.413336277008057 

2번째 가게


...

https://map.naver.com/p/search/%EA%B0%95%EB%8F%99%EA%B5%AC%208%EB%B2%88%EA%B0%80?c=15.00,0,0,0,dh
결과 여러 개 존재(지점 또는 근접 지역)

[크롤링 시작...]
8번가
맥주,호프
서울 강동구 천호대로157길 18 원방빌딩 3층
주문 유형 3
8번가빠네치킨
8번가의 대표메뉴, 로제소스와 치킨의 조화와 치즈가 듬뿍 들어있는 빵을 함께 드실 수 있는 메뉴
리뷰 존재
음식이 맛있어요
대화하기 좋아요
기본 안주가 좋아요
술이 다양해요
인테리어가 멋져요
['8번가'] ...완료
[데이터 수집 완료]
소요 시간 : 13.40165638923645 

81번째 가게


The chromedriver version (128.0.6613.137) detected in PATH at c:\Users\kk21\Desktop\TRIPER\chromedriver.exe might not be compatible with the detected chrome version (129.0.6668.101); currently, chromedriver 129.0.6668.100 is recommended for chrome 129.*, so it is advised to delete the driver in PATH and retry


https://map.naver.com/p/search/%EA%B0%95%EB%8F%99%EA%B5%AC%20%ED%97%A4%EB%A5%B4%EC%B8%A0%EB%A1%9C%EA%B7%B8/place/1976143426?c=15.00,0,0,0,dh&isCorrectAnswer=true
결과 한 개

[크롤링 시작...]
헤르츠로그
카페
서울 강동구 천호옛길 57 연재빌딩 1층 101호
주문 유형 3
헤르츠 에이드
향긋한 캐모마일티와 수제 레몬청, 허브로 이루어진 헤르츠 로그만의 음료입니다^^
리뷰 존재
디저트가 맛있어요
친절해요
특별한 메뉴가 있어요
커피가 맛있어요
음료가 맛있어요
['헤르츠로그'] ...완료
[데이터 수집 완료]
소요 시간 : 13.393106460571289 

82번째 가게


The chromedriver version (128.0.6613.137) detected in PATH at c:\Users\kk21\Desktop\TRIPER\chromedriver.exe might not be compatible with the detected chrome version (129.0.6668.101); currently, chromedriver 129.0.6668.100 is recommended for chrome 129.*, so it is advised to delete the driver in PATH and retry


https://map.naver.com/p/search/%EA%B0%95%EB%8F%99%EA%B5%AC%20%EC%95%94%EC%82%AC%20%ED%95%B4%EC%9E%A5%EA%B5%AD?c=14.00,0,0,0,dh
결과 여러 개 존재(지점 또는 근접 지역)

[크롤링 시작...]
암사고바우감자탕
감자탕
서울 강동구 올림픽로 812
주문 유형 3
뼈다귀탕
장시간 물어 담거 핏물을빼고 푹끓여 부드러운 살고기와 구수한 들깨로 맛 칼칼한맛과 구수한맛이 일품은 해장국
리뷰 존재
음식이 맛있어요
양이 많아요
친절해요
가성비가 좋아요
재료가 신선해요
['암사고바우감자탕'] ...완료
[데이터 수집 완료]
소요 시간 : 13.41345500946045 

./out_food_review/out_24_food.csv완료


In [77]:
#지금 번호 확인
print(i)
print(j)

15
19


In [67]:
# # 합쳐야 할 때 맨 처음 한 번만 실행 
# 지역 4 건너 뜀
ex = tmp.copy()
len(ex)
#ex

22

In [69]:
ex2 = tmp.copy()
len(ex2)

71

In [70]:
import pandas as pd
tmp_df = pd.concat([ex,ex2], axis=0, ignore_index=True)
#tmp_df[-3:]
len(tmp_df)

93

In [72]:
ex = tmp_df.copy()
len(ex)

93

In [73]:
filename = "./out_food_review/out_" + str(8) + "_food.csv"
ex.to_csv(filename, header = None, index = False)
print(filename + "완료")

./out_food_review/out_8_food.csv완료
